In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [44]:

import sys
import sklearn
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import io



       emotion                                             pixels        Usage
0            0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...     Training
1            0  151 150 147 155 148 133 111 140 170 174 182 15...     Training
2            2  231 212 156 164 174 138 161 173 182 200 106 38...     Training
3            4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...     Training
4            6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...     Training
...        ...                                                ...          ...
35882        6  50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...  PrivateTest
35883        3  178 174 172 173 181 188 191 194 196 199 200 20...  PrivateTest
35884        0  17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...  PrivateTest
35885        3  30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...  PrivateTest
35886        2  19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...  PrivateTest

[35887 rows x 3 columns]


In [6]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D, Activation, BatchNormalization, SeparableConv2D,MaxPooling2D,GlobalAveragePooling2D
from keras import Model
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.utils import np_utils
from keras.regularizers import l2
from keras.layers import Input

Using TensorFlow backend.


In [48]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/fer2013.csv')
print(data)
data = data.loc[ data['emotion'].isin([0,3,4,6])]
print(data)
data.loc[(data['emotion'] == 3),'emotion'] = 1
data.loc[(data['emotion'] == 4),'emotion'] = 2
data.loc[(data['emotion'] == 6),'emotion'] = 3
print(data)


       emotion                                             pixels        Usage
0            0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...     Training
1            0  151 150 147 155 148 133 111 140 170 174 182 15...     Training
2            2  231 212 156 164 174 138 161 173 182 200 106 38...     Training
3            4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...     Training
4            6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...     Training
...        ...                                                ...          ...
35882        6  50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...  PrivateTest
35883        3  178 174 172 173 181 188 191 194 196 199 200 20...  PrivateTest
35884        0  17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...  PrivateTest
35885        3  30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...  PrivateTest
35886        2  19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...  PrivateTest

[35887 rows x 3 columns]
       emotion            

In [51]:

train = data.loc[ data['Usage'] == "Training"]
test = data[data.Usage == "PrivateTest"]
test_pub = data[data.Usage == "PublicTest"]

train_labels = train["emotion"].values
test_labels = test["emotion"].values
test_pub_labels = test_pub["emotion"].values

train_ds = train["pixels"].values
test_ds = test["pixels"].values
test_pub_ds = test_pub["pixels"].values

train_ds_list = list()
test_ds_list = list()
test_pub_ds_list = list()
print(train_ds.size)
print(test_ds.size)
print(test_pub_ds.size)
for i in range(train_ds.size):
  train_ds_list.append(np.array(train_ds[i].split(" ")))
  train_ds_list[i] = train_ds_list[i].astype(np.float)

for i in range(test_ds.size):
  test_ds_list.append(np.array(test_ds[i].split(" ")))
  test_ds_list[i] = test_ds_list[i].astype(np.float)

for i in range(test_pub_ds.size):
  test_pub_ds_list.append(np.array(test_pub_ds[i].split(" ")))
  test_pub_ds_list[i] = test_pub_ds_list[i].astype(np.float)

train_np = np.array(train_ds_list)
test_np = np.array(test_ds_list)
test_pub_np = np.array(test_pub_ds_list)



train_np = train_np.reshape(train_np.shape[0], 48, 48, 1)
test_np = test_np.reshape(test_np.shape[0],48, 48, 1)
test_pub_np = test_pub_np.reshape(test_pub_np.shape[0], 48, 48, 1)

train_labels = np_utils.to_categorical(train_labels, 4)
test_labels = np_utils.to_categorical(test_labels, 4)
test_pub_labels = np_utils.to_categorical(test_pub_labels, 4) 

patience = 50
batch_size = 64
num_epochs = 60


train_big = np.vstack([train_np, test_pub_np])
train_label_big = np.concatenate([train_labels, test_pub_labels])

21005
2590
2622


In [52]:
print(train_np.shape)

(21005, 48, 48, 1)


In [0]:
def mini_XCEPTION(input_shape, num_classes, l2_regularization=0.01):
    regularization = l2(l2_regularization)

    # base
    img_input = Input(input_shape)
    x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization,
                                            use_bias=False)(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization,
                                            use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # module 1
    residual = Conv2D(16, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(16, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(16, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # module 2
    residual = Conv2D(32, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(32, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(32, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # module 3
    residual = Conv2D(64, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(64, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(64, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # module 4
    residual = Conv2D(128, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(128, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(128, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    x = Conv2D(num_classes, (3, 3),
            #kernel_regularizer=regularization,
            padding='same')(x)
    x = GlobalAveragePooling2D()(x)
    output = Activation('softmax',name='predictions')(x)

    model = Model(img_input, output)
    return model

In [0]:
num_epochs = 100
early_stop = EarlyStopping('val_loss', patience=patience)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1,
                                  patience=int(patience/4), verbose=1)
model_names ='weights_mini_xception.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,
                                                    save_best_only=True)
callbacks = [model_checkpoint, early_stop, reduce_lr]

In [0]:
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)

In [0]:
model = mini_XCEPTION((48, 48, 1), 4)
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [61]:
model.fit_generator(data_generator.flow(train_big, train_label_big,
                                            batch_size),
                        steps_per_epoch=len(train_big) / batch_size,
                        epochs=100, verbose=1, callbacks=callbacks,
                        validation_data=[test_np,test_labels])
model.save_weights("weights_100_epochs_xception.hdf5")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
370/369 [==============================] - 34s 91ms/step - loss: 1.3843 - acc: 0.4002 - val_loss: 1.3961 - val_acc: 0.4293

Epoch 00001: val_loss improved from inf to 1.39607, saving model to weights_mini_xception.01-0.43.hdf5
Epoch 2/100
370/369 [==============================] - 25s 68ms/step - loss: 1.2027 - acc: 0.4924 - val_loss: 1.5277 - val_acc: 0.4313

Epoch 00002: val_loss did not improve from 1.39607
Epoch 3/100
370/369 [==============================] - 25s 67ms/step - loss: 1.0980 - acc: 0.5424 - val_loss: 1.0684 - val_acc: 0.5722

Epoch 00003: val_loss improved from 1.39607 to 1.06844, saving model to weights_mini_xception.03-0.57.hdf5
Epoch 4/100
370/369 [==============================] - 25s 68ms/step - loss: 1.0390 - acc: 0.5705 - val_loss: 1.0351 - val_acc: 0.5799

Epoch 00004: val_loss improved from 1.06844 to 1.03513, saving model to weights_mini_xception.04-0.

In [69]:
from google.colab import files
from keras.preprocessing import image
import cv2 


uploaded = files.upload()

path = '/content/Happyguy.jpg'
img = image.load_img(path, target_size=(48,48),color_mode='grayscale')
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

classes = model.predict(x)

Saving Happyguy.jpg to Happyguy (3).jpg


In [72]:
print(classes*100)

[[8.2684001e-03 9.9991379e+01 1.4409720e-04 2.0545794e-04]]


In [0]:
model.save('mini_xception_4classes_model.hdf5')